# Run Pipeline on htRNA-Seq Samples

# Gather Results

In [1]:
import os, shutil, sys
from datetime import datetime
import pandas as pd
import numpy as np
from tqdm import tqdm
sys.path.append('/n/boslfs02/LABS/sfortune_lab/Lab/culviner/bin/mtbvartools/')
import mtbvartools as vt

def collectResults(target_directory, target_processes):
    name_list = pd.read_csv(
        target_processes, index_col=0).index
    completed_list = []
    error_list = []
    unhandled_list = []
    completed = 0
    caught_error = 0
    unhandled_error = 0
    for name in tqdm(name_list):
        try:  # get completed timings
            completed_list.append(
                pd.read_csv(f'{target_directory}/{name}/results/{name}.results.csv', index_col=0).loc[name])
            completed += 1
        except FileNotFoundError:
            try:
                error_list.append(
                    pd.read_csv(f'{target_directory}/{name}/results/{name}.error.results.csv', index_col=0).loc[name])
                caught_error += 1
            except FileNotFoundError:
                unhandled_error += 1
                unhandled_list.append(name)
    completed_results = pd.concat(
        completed_list, axis=1).T
    try:
        error_results = pd.concat(
            error_list, axis=1).T
    except:
        error_results = None
    print(f'{completed} completed, {caught_error} caught errors, {unhandled_error} unhandled errors.')
    return completed_results, error_results, unhandled_list

In [2]:
target_directory = 'scratch/variant_pipeline/'
target_processes = 'htrnaseq_strain_pipeline_inputs.csv'

In [3]:
completed_results, error_results, unhandled_list = collectResults(
    target_directory,
    target_processes)
completed_results.to_csv('pipeline_results.csv')

100%|██████████| 274/274 [00:09<00:00, 28.58it/s]


274 completed, 0 caught errors, 0 unhandled errors.


# Merge VCF

## Run Merge Code

In [4]:
import dendropy, os, subprocess, shutil
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [5]:
work_dir = 'scratch/merged_vcf/'
os.makedirs(work_dir, exist_ok=True)

results_inputs = pd.read_csv(
    'pipeline_results.csv', index_col=0)

In [6]:
datasheet_data = [[
    'canettii',
    'SRR10522783/results/SRR10522783.breseq.vcf',
    'SRR10522783/results/SRR10522783.miss.breseq.zarr']]

for index, rdata in results_inputs.iterrows():
    row_data = [
        index,
        f'{index}/results/{index}.breseq.vcf',
        f'{index}/results/{index}.miss.breseq.zarr']
    datasheet_data.append(row_data)

merge_df = pd.DataFrame(
    data=datasheet_data,
    columns=['label', 'vcf_path', 'miss_path'])

# run on all strains
merge_df.to_csv(f'{work_dir}/merge_inputs.csv')

Merge on all strains.

In [7]:
cmd = f'\
    sbatch -c 5 -p hsph -t 2:00:00 --mem=10G -o {work_dir}/merge-%j.out --wrap="\
    module load Mambaforge/22.11.1-fasrc01 && conda activate mtb_isolates && \
    export PATH=\"/n/boslfs02/LABS/sfortune_lab/Lab/culviner/bin/mtbvartools/scripts:/n/home12/pculviner/.conda/envs/mtb_isolates/bin/:$PATH\" && \
    export PYTHONPATH=\"${{PYTHONPATH}}:/n/boslfs02/LABS/sfortune_lab/Lab/culviner/bin/mtbvartools\" && \
    merge_vcfs_misses.py \
    --input-csv {work_dir}/merge_inputs.csv \
    --inputs-dir scratch/variant_pipeline/ \
    --out-dir {work_dir} \
    --outgroup canettii \
    --split-genome 2 \
    --n-workers 25 \
    --process-per-node 10 \
    --walltime "1:00:00" \
    --use-slurm \
    --local-threads 5"'
subprocess.run(cmd, shell=True)

Submitted batch job 52202095


CompletedProcess(args='    sbatch -c 5 -p hsph -t 2:00:00 --mem=10G -o scratch/merged_vcf//merge-%j.out --wrap="    module load Mambaforge/22.11.1-fasrc01 && conda activate mtb_isolates &&     export PATH="/n/boslfs02/LABS/sfortune_lab/Lab/culviner/bin/mtbvartools/scripts:/n/home12/pculviner/.conda/envs/mtb_isolates/bin/:$PATH" &&     export PYTHONPATH="${PYTHONPATH}:/n/boslfs02/LABS/sfortune_lab/Lab/culviner/bin/mtbvartools" &&     merge_vcfs_misses.py     --input-csv scratch/merged_vcf//merge_inputs.csv     --inputs-dir scratch/variant_pipeline/     --out-dir scratch/merged_vcf/     --outgroup canettii     --split-genome 2     --n-workers 25     --process-per-node 10     --walltime "1:00:00"     --use-slurm     --local-threads 5"', returncode=0)

## Generate Annotation Files

In [9]:
import dendropy, os, subprocess, shutil, tqdm, sys
import pandas as pd
import numpy as np
sys.path.append('/n/boslfs02/LABS/sfortune_lab/Lab/culviner/bin/mtbvartools/')
import mtbvartools as vt
from mtbvartools.dasktools import startClient, subproc
from mtbvartools.vcf import filterDataFrame

# gene table
gene_table = pd.read_csv('gt_mtb_h37rv.csv', index_col=0)

work_dir = 'scratch/merged_vcf/'
os.makedirs(work_dir, exist_ok=True)

In [10]:
cmd = f'\
    sbatch -c 2 -p hsph -t 2:00:00 --mem=10G -o {work_dir}/anno1-%j.out --wrap=" \
    module load Mambaforge/22.11.1-fasrc01 && conda activate mtb_isolates && \
    cd {work_dir} && bcftools view -s SRR10522783 merged.filtered.vcf.gz | \
    bcftools norm -m - -O z -o annotations_input.vcf.gz && tabix annotations_input.vcf.gz"'
subprocess.run(cmd, shell=True)

Submitted batch job 52203035


CompletedProcess(args='    sbatch -c 2 -p hsph -t 2:00:00 --mem=10G -o scratch/merged_vcf//anno1-%j.out --wrap="     module load Mambaforge/22.11.1-fasrc01 && conda activate mtb_isolates &&     cd scratch/merged_vcf/ && bcftools view -s SRR10522783 merged.filtered.vcf.gz |     bcftools norm -m - -O z -o annotations_input.vcf.gz && tabix annotations_input.vcf.gz"', returncode=0)

In [11]:
cmd = f'\
    sbatch -c 4 -p hsph -t 2:00:00 --mem=10G -o {work_dir}/anno2-%j.out --wrap=" \
    module load Mambaforge/22.11.1-fasrc01 && conda activate mtb_isolates && \
    cd {work_dir} && bcftools view annotations_input.vcf.gz | \
    awk \'{{gsub(\\"NC_000962\\",\\"Chromosome\\"); print}}\' | \
    snpEff -ud 500 -v \
    -configOption Mycobacterium_tuberculosis_h37rv.codonTable=Bacterial_and_Plant_Plastid Mycobacterium_tuberculosis_h37rv > \
    snpeff_annotated.tmp.vcf"'
subprocess.run(cmd, shell=True)

Submitted batch job 52203138


CompletedProcess(args='    sbatch -c 4 -p hsph -t 2:00:00 --mem=10G -o scratch/merged_vcf//anno2-%j.out --wrap="     module load Mambaforge/22.11.1-fasrc01 && conda activate mtb_isolates &&     cd scratch/merged_vcf/ && bcftools view annotations_input.vcf.gz |     awk \'{gsub(\\"NC_000962\\",\\"Chromosome\\"); print}\' |     snpEff -ud 500 -v     -configOption Mycobacterium_tuberculosis_h37rv.codonTable=Bacterial_and_Plant_Plastid Mycobacterium_tuberculosis_h37rv >     snpeff_annotated.tmp.vcf"', returncode=0)

In [12]:
sift_path = '/n/boslfs02/LABS/sfortune_lab/Lab/culviner/sift/'
cmd = f'\
    sbatch -c 4 -p hsph -t 2:00:00 --mem=10G -o {work_dir}/anno3-%j.out --wrap=" \
    module load Mambaforge/22.11.1-fasrc01 && conda activate mtb_isolates && \
    cd {work_dir} && java -jar  {sift_path}/SIFT4G_Annotator.jar \
    -c -i snpeff_annotated.tmp.vcf -d {sift_path}/GCA_000195955.2.22 -r sift_results -t && \
    awk \'{{gsub(\\"Chromosome\\",\\"NC_000962.3\\"); print}}\' \
    sift_results/snpeff_annotated.tmp_SIFTpredictions.vcf | \
    bcftools view -O z -o annotated.vcf.gz && tabix annotated.vcf.gz"'
subprocess.run(cmd, shell=True)

Submitted batch job 52203162


CompletedProcess(args='    sbatch -c 4 -p hsph -t 2:00:00 --mem=10G -o scratch/merged_vcf//anno3-%j.out --wrap="     module load Mambaforge/22.11.1-fasrc01 && conda activate mtb_isolates &&     cd scratch/merged_vcf/ && java -jar  /n/boslfs02/LABS/sfortune_lab/Lab/culviner/sift//SIFT4G_Annotator.jar     -c -i snpeff_annotated.tmp.vcf -d /n/boslfs02/LABS/sfortune_lab/Lab/culviner/sift//GCA_000195955.2.22 -r sift_results -t &&     awk \'{gsub(\\"Chromosome\\",\\"NC_000962.3\\"); print}\'     sift_results/snpeff_annotated.tmp_SIFTpredictions.vcf |     bcftools view -O z -o annotated.vcf.gz && tabix annotated.vcf.gz"', returncode=0)

In [13]:
client = startClient(
    n_workers=25,
    use_local=False,
    use_slurm=True,
    log_dir='/n/home12/pculviner/logs/',
    queue='sapphire',
    process_per_node=5,
    cores_per_process=1,
    memory_per_process='4GB',
    walltime='2:00:00',
    dashboard_address=':10000',
    job_script_prologue=[
        'export PYTHONPATH="${PYTHONPATH}:/n/boslfs02/LABS/sfortune_lab/Lab/culviner/bin/mtbvartools/"'])

# for parallel proc
@subproc
def annojob(geneid, left, right):
    return vt.fetchGeneVariantAnnotations(
        f'{work_dir}/annotated.vcf.gz', geneid, left, right)

futures = []
for geneid, gdata in gene_table.iterrows():
    futures.append(client.submit(annojob, geneid, gdata.start + 1, gdata.end + 1))

output_df = []
for f in tqdm.tqdm(futures):
    output_df.append(client.gather(f))
    f.release()
output_df = pd.concat(output_df, axis=0)
client.shutdown()

output_df.to_csv(
    f'{work_dir}/vcf_annotations.csv')

output_df = pd.read_csv(
    f'{work_dir}/vcf_annotations.csv')

# filter
deleterious_settings = {
    'Annotation': lambda x: any([atype in str(x) for atype in [
        'frameshift_variant',
        'stop_gained',
        'start_lost']]),
    'SIFT_score': lambda x: float(x) < 0.05}

deleterious_df = filterDataFrame(output_df, function_dict=deleterious_settings)
deleterious_df.to_csv(f'{work_dir}/vcf_annotations.deleterious.csv')

2024-10-17 12:42:48 - Launching SLURMCluster client (25 workers x 1 CPU x 4GB x 2:00:00 @ sapphire with 5 workers / node)


100%|██████████| 3974/3974 [02:51<00:00, 23.11it/s]  


# Run Tree Construction

## Generate Fastas

**SNP fasta**

In [17]:
import dendropy, os, subprocess, shutil
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

work_dir = 'scratch/snp_fasta/'
os.makedirs(work_dir, exist_ok=True)

results_inputs = pd.read_csv(
    'pipeline_results.csv', index_col=0)

In [18]:
datasheet_data = [[
    'canettii',
    'SRR10522783/results/SRR10522783.breseq.vcf',
    'SRR10522783/results/SRR10522783.miss.breseq.zarr']]

for index, rdata in results_inputs.iterrows():
    row_data = [
        index,
        f'{index}/results/{index}.breseq.vcf',
        f'{index}/results/{index}.miss.breseq.zarr']
    datasheet_data.append(row_data)

tree_df = pd.DataFrame(
    data=datasheet_data,
    columns=['label', 'vcf_path', 'miss_path'])

# run on all strains
tree_df.to_csv(f'{work_dir}/tree_inputs.csv')

# run fasta generation command for supertree
cmd = f'\
    sbatch -c 20 -p sapphire -t 2:00:00 --mem=40G -o {work_dir}/fasta-%j.out --wrap="\
    module load Mambaforge/22.11.1-fasrc01 && conda activate mtb_isolates && \
    export PATH=\"/n/boslfs02/LABS/sfortune_lab/Lab/culviner/bin/mtbvartools/scripts:/n/home12/pculviner/.conda/envs/mtb_isolates/bin/:$PATH\" && \
    export PYTHONPATH=\"${{PYTHONPATH}}:/n/boslfs02/LABS/sfortune_lab/Lab/culviner/bin/mtbvartools\" && \
    write_snp_fastas.py \
    --input-csv {work_dir}/tree_inputs.csv \
    --input-fasta pipeline_inputs/Mtb_h37rv.fasta \
    --inputs-dir scratch/variant_pipeline/ \
    --out-dir {work_dir} \
    --output snp_fasta \
    --mask rlc_plus_lowmap_marin.bed \
    --miss-threshold 0.1 \
    --local-threads 20"'
subprocess.run(cmd, shell=True)

Submitted batch job 52224742


CompletedProcess(args='    sbatch -c 20 -p sapphire -t 2:00:00 --mem=40G -o scratch/snp_fasta//fasta-%j.out --wrap="    module load Mambaforge/22.11.1-fasrc01 && conda activate mtb_isolates &&     export PATH="/n/boslfs02/LABS/sfortune_lab/Lab/culviner/bin/mtbvartools/scripts:/n/home12/pculviner/.conda/envs/mtb_isolates/bin/:$PATH" &&     export PYTHONPATH="${PYTHONPATH}:/n/boslfs02/LABS/sfortune_lab/Lab/culviner/bin/mtbvartools" &&     write_snp_fastas.py     --input-csv scratch/snp_fasta//tree_inputs.csv     --input-fasta pipeline_inputs/Mtb_h37rv.fasta     --inputs-dir scratch/variant_pipeline/     --out-dir scratch/snp_fasta/     --output snp_fasta     --mask rlc_plus_lowmap_marin.bed     --miss-threshold 0.1     --local-threads 20"', returncode=0)

**genome fasta**

In [20]:
import dendropy, os, subprocess, shutil
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

work_dir = 'scratch/genome_fasta/'
os.makedirs(work_dir, exist_ok=True)

results_inputs = pd.read_csv(
    'pipeline_results.csv', index_col=0)

In [21]:
datasheet_data = [[
    'canettii',
    'SRR10522783/results/SRR10522783.breseq.vcf',
    'SRR10522783/results/SRR10522783.miss.breseq.zarr']]

for index, rdata in results_inputs.iterrows():
    row_data = [
        index,
        f'{index}/results/{index}.breseq.vcf',
        f'{index}/results/{index}.miss.breseq.zarr']
    datasheet_data.append(row_data)

tree_df = pd.DataFrame(
    data=datasheet_data,
    columns=['label', 'vcf_path', 'miss_path'])

# run on all strains
tree_df.to_csv(f'{work_dir}/tree_inputs.csv')

# run fasta generation command for supertree
cmd = f'\
    sbatch -c 20 -p sapphire -t 2:00:00 --mem=40G -o {work_dir}/fasta-%j.out --wrap="\
    module load Mambaforge/22.11.1-fasrc01 && conda activate mtb_isolates && \
    export PATH=\"/n/boslfs02/LABS/sfortune_lab/Lab/culviner/bin/mtbvartools/scripts:/n/home12/pculviner/.conda/envs/mtb_isolates/bin/:$PATH\" && \
    export PYTHONPATH=\"${{PYTHONPATH}}:/n/boslfs02/LABS/sfortune_lab/Lab/culviner/bin/mtbvartools\" && \
    write_full_fastas.py \
    --input-csv {work_dir}/tree_inputs.csv \
    --input-fasta pipeline_inputs/Mtb_h37rv.fasta \
    --inputs-dir scratch/variant_pipeline/ \
    --out-dir {work_dir} \
    --output genome_fasta \
    --mask rlc_plus_lowmap_marin.bed \
    --miss-threshold 0.1 \
    --local-threads 20"'
subprocess.run(cmd, shell=True)

Submitted batch job 52225193


CompletedProcess(args='    sbatch -c 20 -p sapphire -t 2:00:00 --mem=40G -o scratch/genome_fasta//fasta-%j.out --wrap="    module load Mambaforge/22.11.1-fasrc01 && conda activate mtb_isolates &&     export PATH="/n/boslfs02/LABS/sfortune_lab/Lab/culviner/bin/mtbvartools/scripts:/n/home12/pculviner/.conda/envs/mtb_isolates/bin/:$PATH" &&     export PYTHONPATH="${PYTHONPATH}:/n/boslfs02/LABS/sfortune_lab/Lab/culviner/bin/mtbvartools" &&     write_full_fastas.py     --input-csv scratch/genome_fasta//tree_inputs.csv     --input-fasta pipeline_inputs/Mtb_h37rv.fasta     --inputs-dir scratch/variant_pipeline/     --out-dir scratch/genome_fasta/     --output genome_fasta     --mask rlc_plus_lowmap_marin.bed     --miss-threshold 0.1     --local-threads 20"', returncode=0)

## Construct Trees

In [1]:
import dendropy, os, subprocess, shutil
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

work_dir = 'scratch/trees/'
os.makedirs(work_dir, exist_ok=True)

### fasttree SNPs, rawdist

In [25]:
tree_label = 'ft2_snps_rawdist'
input_fasta = 'scratch/snp_fasta/snp_fasta.fasta'
os.makedirs(f'{work_dir}/{tree_label}', exist_ok=True)
threads = 25
memory = 25

cmd = f'\
    sbatch -c {threads} -p sapphire -t 12:00:00 --mem={memory}G -o {work_dir}/{tree_label}/tree-%j.out --wrap="\
    export OMP_NUM_THREADS={threads} && \
    FastTreeMP -rawdist -nt {input_fasta} > {work_dir}/{tree_label}/{tree_label}.nwk"'
subprocess.run(cmd, shell=True)

Submitted batch job 52226356


CompletedProcess(args='    sbatch -c 25 -p sapphire -t 12:00:00 --mem=25G -o scratch/trees//ft2_snps_rawdist/tree-%j.out --wrap="    export OMP_NUM_THREADS=25 &&     FastTreeMP -rawdist -nt scratch/snp_fasta/snp_fasta.fasta > scratch/trees//ft2_snps_rawdist/ft2_snps_rawdist.nwk"', returncode=0)

### fasttree SNPs, rawdist

In [25]:
tree_label = 'ft2_snps_rawdist'
input_fasta = 'scratch/snp_fasta/snp_fasta.fasta'
os.makedirs(f'{work_dir}/{tree_label}', exist_ok=True)
threads = 25
memory = 25

cmd = f'\
    sbatch -c {threads} -p sapphire -t 12:00:00 --mem={memory}G -o {work_dir}/{tree_label}/tree-%j.out --wrap="\
    export OMP_NUM_THREADS={threads} && \
    FastTreeMP -rawdist -nt {input_fasta} > {work_dir}/{tree_label}/{tree_label}.nwk"'
subprocess.run(cmd, shell=True)

Submitted batch job 52226356


CompletedProcess(args='    sbatch -c 25 -p sapphire -t 12:00:00 --mem=25G -o scratch/trees//ft2_snps_rawdist/tree-%j.out --wrap="    export OMP_NUM_THREADS=25 &&     FastTreeMP -rawdist -nt scratch/snp_fasta/snp_fasta.fasta > scratch/trees//ft2_snps_rawdist/ft2_snps_rawdist.nwk"', returncode=0)

### fasttree SNPs, GTR

In [26]:
tree_label = 'ft2_snps_gtr'
input_fasta = 'scratch/snp_fasta/snp_fasta.fasta'
os.makedirs(f'{work_dir}/{tree_label}', exist_ok=True)
threads = 25
memory = 25

cmd = f'\
    sbatch -c {threads} -p sapphire -t 12:00:00 --mem={memory}G -o {work_dir}/{tree_label}/tree-%j.out --wrap="\
    export OMP_NUM_THREADS={threads} && \
    FastTreeMP -gtr -nt {input_fasta} > {work_dir}/{tree_label}/{tree_label}.nwk"'
subprocess.run(cmd, shell=True)

Submitted batch job 52226468


CompletedProcess(args='    sbatch -c 25 -p sapphire -t 12:00:00 --mem=25G -o scratch/trees//ft2_snps_gtr/tree-%j.out --wrap="    export OMP_NUM_THREADS=25 &&     FastTreeMP -gtr -nt scratch/snp_fasta/snp_fasta.fasta > scratch/trees//ft2_snps_gtr/ft2_snps_gtr.nwk"', returncode=0)

### iqtree SNPs gtr

In [27]:
tree_label = 'iq_snp_gtr'
input_fasta = 'scratch/snp_fasta/snp_fasta.fasta'
os.makedirs(f'{work_dir}/{tree_label}', exist_ok=True)
threads = 25
memory = 25

cmd = f'\
    sbatch -c {threads} -p sapphire -t 24:00:00 --mem={memory}G -o {work_dir}/{tree_label}/tree-%j.out --wrap="\
    export OMP_NUM_THREADS={threads} && \
    module load Mambaforge/22.11.1-fasrc01 && conda activate mtb_isolates && \
    iqtree -T {threads} -s {input_fasta} --model GTR -bb 1000 --prefix {work_dir}/{tree_label}/{tree_label}"'
subprocess.run(cmd, shell=True)

Submitted batch job 52226874


CompletedProcess(args='    sbatch -c 25 -p sapphire -t 24:00:00 --mem=25G -o scratch/trees//iq_snp_gtr/tree-%j.out --wrap="    export OMP_NUM_THREADS=25 &&     module load Mambaforge/22.11.1-fasrc01 && conda activate mtb_isolates &&     iqtree -T 25 -s scratch/snp_fasta/snp_fasta.fasta --model GTR -bb 1000 --prefix scratch/trees//iq_snp_gtr/iq_snp_gtr"', returncode=0)

### iqtree SNPs gtr constants
- Use this one for downstream analyses

In [28]:
tree_label = 'iq_snp_gtr_cons'
input_fasta = 'scratch/snp_fasta/snp_fasta.fasta'
os.makedirs(f'{work_dir}/{tree_label}', exist_ok=True)
# added A, C, G, T
threads = 25
memory = 25

cmd = f'\
    sbatch -c {threads} -p sapphire -t 24:00:00 --mem={memory}G -o {work_dir}/{tree_label}/tree-%j.out --wrap="\
    export OMP_NUM_THREADS={threads} && \
    module load Mambaforge/22.11.1-fasrc01 && conda activate mtb_isolates && \
    iqtree -T {threads} -fconst 700494,1322616,1318890,700844 -s {input_fasta} --model GTR -bb 1000 --prefix {work_dir}/{tree_label}/{tree_label}"'
subprocess.run(cmd, shell=True)

Submitted batch job 52235431


CompletedProcess(args='    sbatch -c 25 -p sapphire -t 24:00:00 --mem=25G -o scratch/trees//iq_snp_gtr_cons/tree-%j.out --wrap="    export OMP_NUM_THREADS=25 &&     module load Mambaforge/22.11.1-fasrc01 && conda activate mtb_isolates &&     iqtree -T 25 -fconst 700494,1322616,1318890,700844 -s scratch/snp_fasta/snp_fasta.fasta --model GTR -bb 1000 --prefix scratch/trees//iq_snp_gtr_cons/iq_snp_gtr_cons"', returncode=0)

In [2]:
tree_label = 'iq_snp_gtr_cons'
# load the tree
tree = dendropy.Tree.get_from_path(
    f'{work_dir}/{tree_label}/{tree_label}.contree', 'newick')
# root at canettii
tree.reroot_at_node(
    tree.find_node_with_taxon_label('canettii'))
tree.prune_taxa_with_labels('canettii')
# ladderize
tree.ladderize()
tree.write(
    path=f'{work_dir}/{tree_label}/{tree_label}.rooted.contree.nwk',
    schema='newick')

In [3]:
tree_label = 'iq_snp_gtr_cons'
# load the tree
tree = dendropy.Tree.get_from_path(
    f'{work_dir}/{tree_label}/{tree_label}.contree', 'newick')
# root at canettii
tree.reroot_at_node(
    tree.find_node_with_taxon_label('canettii').parent_node)
# ladderize
tree.ladderize()
tree.write(
    path=f'{work_dir}/{tree_label}/{tree_label}.outgroup.contree.nwk',
    schema='newick')

### iqtree SNPs pathogen

In [3]:
tree_label = 'iq_snp_pathogen'
input_fasta = 'scratch/snp_fasta/snp_fasta.fasta'
os.makedirs(f'{work_dir}/{tree_label}', exist_ok=True)
threads = 25
memory = 25

cmd = f'\
    sbatch -c {threads} -p sapphire -t 24:00:00 --mem={memory}G -o {work_dir}/{tree_label}/tree-%j.out --wrap="\
    export OMP_NUM_THREADS={threads} && \
    module load Mambaforge/22.11.1-fasrc01 && conda activate mtb_isolates && \
    iqtree -T {threads} -s {input_fasta} --pathogen-force --model GTR -bb 1000 --prefix {work_dir}/{tree_label}/{tree_label}"'
subprocess.run(cmd, shell=True)

Submitted batch job 52305775


CompletedProcess(args='    sbatch -c 25 -p sapphire -t 24:00:00 --mem=25G -o scratch/trees//iq_snp_pathogen/tree-%j.out --wrap="    export OMP_NUM_THREADS=25 &&     module load Mambaforge/22.11.1-fasrc01 && conda activate mtb_isolates &&     iqtree -T 25 -s scratch/snp_fasta/snp_fasta.fasta --pathogen-force --model GTR -bb 1000 --prefix scratch/trees//iq_snp_pathogen/iq_snp_pathogen"', returncode=0)

### iqtree SNPs pathogen constants

In [4]:
tree_label = 'iq_snp_pathogen_cons'
input_fasta = 'scratch/snp_fasta/snp_fasta.fasta'
os.makedirs(f'{work_dir}/{tree_label}', exist_ok=True)
threads = 25
memory = 25
cmd = f'\
    sbatch -c {threads} -p sapphire -t 24:00:00 --mem={memory}G -o {work_dir}/{tree_label}/tree-%j.out --wrap="\
    export OMP_NUM_THREADS={threads} && \
    module load Mambaforge/22.11.1-fasrc01 && conda activate mtb_isolates && \
    iqtree -T {threads} -s {input_fasta} -fconst 700494,1322616,1318890,700844 --pathogen-force --model GTR -bb 1000 --prefix {work_dir}/{tree_label}/{tree_label}"'
subprocess.run(cmd, shell=True)

Submitted batch job 52305781


CompletedProcess(args='    sbatch -c 25 -p sapphire -t 24:00:00 --mem=25G -o scratch/trees//iq_snp_pathogen_cons/tree-%j.out --wrap="    export OMP_NUM_THREADS=25 &&     module load Mambaforge/22.11.1-fasrc01 && conda activate mtb_isolates &&     iqtree -T 25 -s scratch/snp_fasta/snp_fasta.fasta -fconst 700494,1322616,1318890,700844 --pathogen-force --model GTR -bb 1000 --prefix scratch/trees//iq_snp_pathogen_cons/iq_snp_pathogen_cons"', returncode=0)

### iqtree genome pathogen constants

In [6]:
tree_label = 'iq_genome_pathogen'
input_fasta = 'scratch/genome_fasta/genome_fasta.fasta'
os.makedirs(f'{work_dir}/{tree_label}', exist_ok=True)
threads = 25
memory = 25
cmd = f'\
    sbatch -c {threads} -p sapphire -t 24:00:00 --mem={memory}G -o {work_dir}/{tree_label}/tree-%j.out --wrap="\
    export OMP_NUM_THREADS={threads} && \
    module load Mambaforge/22.11.1-fasrc01 && conda activate mtb_isolates && \
    iqtree -T {threads} -s {input_fasta} --pathogen-force --model GTR -alrt 1000 --prefix {work_dir}/{tree_label}/{tree_label}"'
subprocess.run(cmd, shell=True)

Submitted batch job 52305814


CompletedProcess(args='    sbatch -c 25 -p sapphire -t 24:00:00 --mem=25G -o scratch/trees//iq_genome_pathogen/tree-%j.out --wrap="    export OMP_NUM_THREADS=25 &&     module load Mambaforge/22.11.1-fasrc01 && conda activate mtb_isolates &&     iqtree -T 25 -s scratch/genome_fasta/genome_fasta.fasta --pathogen-force --model GTR -alrt 1000 --prefix scratch/trees//iq_genome_pathogen/iq_genome_pathogen"', returncode=0)

# Ancestral Reconstruction & Event Calling

In [2]:
import zarr, sys, tqdm, dendropy, time, os
import numpy as np
import pandas as pd
sys.path.append('/n/boslfs02/LABS/sfortune_lab/Lab/culviner/bin/mtbvartools/')
import mtbvartools as vt
from mtbvartools.dasktools import startClient

In [3]:
work_dir = 'scratch/ancres/'
os.makedirs(work_dir, exist_ok=True)

## VCF Conversion

In [5]:
client = startClient(
    n_workers=25,
    use_local=False,
    use_slurm=True,
    log_dir='/n/home12/pculviner/logs/',
    queue='sapphire',
    process_per_node=5,
    cores_per_process=1,
    memory_per_process='5GB',
    walltime='1:00:00',
    dashboard_address=':10000',
    job_script_prologue=[
        'export PYTHONPATH="${PYTHONPATH}:/n/boslfs02/LABS/sfortune_lab/Lab/culviner/bin/mtbvartools/"'])

2024-11-04 12:12:21 - Launching SLURMCluster client (25 workers x 1 CPU x 5GB x 1:00:00 @ sapphire with 5 workers / node)


In [6]:
vt.writeVariantCalls(
    'scratch/merged_vcf/merged.filtered.vcf.gz', f'{work_dir}/241103_variant_calls.vcb',)
time.sleep(5)
client.shutdown()

importing calls from VCF (indexed & compressed by variants)....


100%|██████████| 1000/1000 [00:32<00:00, 30.43it/s]


rechunking compression (for off axis random access)....


100%|██████████| 50/50 [00:26<00:00,  1.87it/s]


## Ancestral Reconstruction

In [7]:
client = startClient(
    n_workers=25,
    use_local=False,
    use_slurm=True,
    log_dir='.',
    queue='sapphire',
    process_per_node=5,
    cores_per_process=1,
    memory_per_process='5GB',
    walltime='1:00:00',
    dashboard_address=':10000',
    job_script_prologue=[
        'export PYTHONPATH="${PYTHONPATH}:/n/boslfs02/LABS/sfortune_lab/Lab/culviner/bin/mtbvartools/"'])

2024-11-04 12:14:20 - Launching SLURMCluster client (25 workers x 1 CPU x 5GB x 1:00:00 @ sapphire with 5 workers / node)


In [8]:
vt.writeAncestorCalls(
    vcb_path='scratch/ancres/241103_variant_calls.vcb',
    tree_path='scratch/trees/iq_snp_gtr_cons/iq_snp_gtr_cons.outgroup.contree.nwk',
    output_path=f'{work_dir}/241103_ancestor_calls.vcb',
    step_size=25, miss_filter=0.75, timeout=300, rechunk_jobs=25)
time.sleep(5)
client.shutdown()

100%|██████████| 1126/1126 [00:33<00:00, 33.30it/s]


rechunking compression (for off axis random access)....


100%|██████████| 25/25 [00:00<00:00, 60.93it/s]


## Event Calling

In [4]:
client = startClient(
    n_workers=3,
    use_local=True)

2024-11-04 12:30:00 - Launching LocalCluster client (3 workers)


In [5]:
vt.writeEventCalls(
    'scratch/ancres/241103_ancestor_calls.vcb', 'scratch/ancres/241103_variant_calls.vcb', 'scratch/ancres/241103_event_calls.vcb',
    annotations_csv='scratch/merged_vcf/vcf_annotations.csv',
    gene_table_csv='pipeline_inputs/gt_mtb_h37rv.csv',
    rechunk_jobs=25)
time.sleep(5)
client.shutdown()

writing transitions....


100%|██████████| 273/273 [00:00<00:00, 1091.25it/s]


rechunking compression (for off axis random access)....


100%|██████████| 25/25 [00:05<00:00,  4.43it/s]


counting reversions....


100%|██████████| 548/548 [00:01<00:00, 529.69it/s]


merging gene-level annotations....
writing genomic annotations....


100%|██████████| 27945/27945 [00:15<00:00, 1759.89it/s]
